In [ ]:
import sys, os, shutil
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, GlobalAveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, RandomFlip, RandomZoom, RandomContrast, RandomRotation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
import kerastuner as kt
from random import random
import pandas as pd
import re
physical_devices = tf.config.experimental.list_physical_devices('GPU')
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(f'Running on Python {sys.version}, Tensorflow {tf.__version__}.')

In [ ]:
# data loading config
batch_size = 32
img_height = 224
img_width = 224
dataPath = 'oxford_flowers_102'
labels = 'inferred'
label_mode = 'categorical'  # one hot encoding
color_mode = 'rgb'
shuffle = True
seed = 69
test_split = 0.2  # split into train and test (NOT val), 0-1
AUTOTUNE = tf.data.AUTOTUNE

train_df = pd.read_csv('image_to_label.csv', names=['picture', 'label'])
with open('102_flower_labels.txt') as f:
    regex = re.compile('[^a-zA-Z\s-]')
    classes=[]
    for line in f:
        classes += [regex.sub('', line.replace('\'', '').strip())]


def split_data(dataPath):
    for root, dirs, files in os.walk(dataPath):
        for name in files:
            randomNum = random()
            row = train_df.iloc[train_df.index[train_df['picture'] == name]]
            if randomNum <= test_split:
                os.makedirs('test\\'+classes[row.label.tolist()[0]-1]+'\\', exist_ok=True)
                shutil.move(root+'\\'+name, 'test\\'+classes[row.label.tolist()[0]-1]+'\\')
            elif test_split< randomNum <= test_split + test_split * (1-test_split):
                os.makedirs('val\\'+classes[row.label.tolist()[0]-1]+'\\', exist_ok=True)
                shutil.move(root+'\\'+name, 'val\\'+classes[row.label.tolist()[0]-1]+'\\')
            else:
                os.makedirs('train\\'+classes[row.label.tolist()[0]-1]+'\\', exist_ok=True)
                shutil.move(root+'\\'+name, 'train\\'+classes[row.label.tolist()[0]-1]+'\\')

# split_data(dataPath)  # Only need to run this once

train_datagen = ImageDataGenerator( # Do 0-1 scaling as a layer so that saved model includes it
    rotation_range=20, width_shift_range=0.2,
    height_shift_range=0.2, brightness_range=(-0.2, 0.2), shear_range=0.2, zoom_range=0.2,
    channel_shift_range=0.2, fill_mode='nearest', horizontal_flip=True, vertical_flip=True)


print('Training data:')
# NOT USING GENERATOR AS IT DOES NOT WORK YET
train_generator = train_datagen.flow_from_directory('train', target_size=(img_height, img_width), batch_size=batch_size,
                                                    color_mode=color_mode, class_mode=label_mode, shuffle=shuffle, seed=seed)
train = keras.preprocessing.image_dataset_from_directory('train', labels=labels, label_mode=label_mode,
color_mode=color_mode, shuffle=shuffle, seed=seed, image_size=(img_height, img_width), batch_size=batch_size)
train_class_names = train.class_names

print('\nValidation data:')
val = keras.preprocessing.image_dataset_from_directory('val', labels=labels, label_mode=label_mode,
color_mode=color_mode, shuffle=shuffle, seed=seed, image_size=(img_height, img_width), batch_size=batch_size)
val_class_names = val.class_names

print('\nTesting data:')
test = keras.preprocessing.image_dataset_from_directory('test', labels=labels, label_mode=label_mode,
color_mode=color_mode, shuffle=shuffle, seed=seed, image_size=(img_height, img_width), batch_size=batch_size)
test_class_names = test.class_names

train = train.cache().prefetch(buffer_size=AUTOTUNE)
val = val.cache().prefetch(buffer_size=AUTOTUNE)
test = test.cache().prefetch(buffer_size=AUTOTUNE)

assert list(train_generator.class_indices.keys()) == train_class_names == val_class_names == test_class_names, 'Classes mismatch!'
classes = list(train_generator.class_indices.keys())
print('\nClasses:', classes)

In [ ]:
# Resnet 50 pretrained
xInput = Input([img_height, img_width, 3], dtype=tf.uint8)
x = tf.cast(xInput, tf.float32)
x = keras.applications.resnet.preprocess_input(x)
resnet50 = tf.keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet')
x = resnet50(x, trainable=False)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(512)(x)
x = BatchNormalization(epsilon=1.001e-5)(x)
x = Activation('relu')(x)
x = Dense(512)(x)
x = BatchNormalization(epsilon=1.001e-5)(x)
x = Activation('relu')(x)
xOutput = Dense(len(classes))(x)  # no activation as loss using logit=True
model = tf.keras.models.Model(xInput, xOutput)

In [ ]:
opt = keras.optimizers.Adam(learning_rate=3e-4, epsilon=1e-6)
opt = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1)
reg = keras.regularizers.L2(1e-5)
epoch = 100
metrics = ['accuracy']
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1,
                                     mode='auto', baseline=None, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('./best_model',monitor='val_accuracy',save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=3, verbose=1)
]
model.compile(optimizer=opt, loss=loss, metrics=metrics)
model.summary()
history = model.fit(train, epochs=epoch, validation_data=val, callbacks=callbacks, use_multiprocessing=True, verbose=1)

In [ ]:
plot_model(model, show_shapes=True, show_dtype=True, show_layer_names=True, to_file='cv_model.png')
model.evaluate(test)
model.save('oxford_flower_102')
